In [1]:
import numpy as np
import pandas as pd
import gc

import matplotlib.pyplot as plt
%matplotlib inline
# plt.figure(figsize=(16, 6))

import warnings
warnings.filterwarnings("ignore")


# Load Data and code

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')



In [3]:
# cd "/content/drive/My Drive/continual-learning"

In [4]:
# !git clone https://github.com/chameleonTK/continual-learning-for-HAR.git src

In [5]:
# cd "src"

In [6]:
# !pip install visdom

In [7]:
#!/usr/bin/env python3
import torch
import numpy as np
from smart_home_dataset import SmartHomeDataset
from classifier import Classifier
from torch import optim
import utils
import callbacks as cb
import time

from generative_replay_learner import GenerativeReplayLearner;
import arg_params
import json
import os
import copy

import torch.multiprocessing as mp

# Get Dataset

In [8]:
def select_dataset(args, classes=None):
    if args.data_dir == "pamap":
        default_classes = [
            'lying', 
            'sitting', 
            'standing', 
            'ironing', 
            'vacuum cleaning', 
            'ascending stairs', 
            'walking', 
            'descending stairs', 
            'cycling', 
            'running'
        ]

        data_dir = "./Dataset/PAMAP2/pamap.feat"

    elif args.data_dir == "dsads":
        default_classes = [
            "sitting",
            "standing",
            "lying on back side",
            "lying on right side",
            "ascending stairs",
            "descending stairs",
            "exercising on a stepper",
            "rowing",
            "jumping",
            "playing basketball"
        ]

        data_dir = "./Dataset/DSADS/dsads.feat"


    elif args.data_dir == "housea":
        default_classes = ['A0', 'A1', 'A2', 'A3', 'A4', 'A5'] #skip A6
        args.tasks = 3
        data_dir = "./Dataset/House/HouseA.feat"

    elif args.data_dir == "casas":
        default_classes = [
            "R1_work_at_computer",
            "R2_work_at_computer",
            "R1_sleep",
            "R2_sleep",
            "R1_bed_to_toilet",
            "R2_bed_to_toilet",

            "R2_prepare_dinner",
            "R2_watch_TV",
            
            "R2_prepare_lunch",
            "R1_work_at_dining_room_table",
        ]
        data_dir = "./Dataset/twor.2009/annotated.feat.ch1"
    else:
        raise Exception("Unknow dataset")
    
    if classes is None:
        classes = default_classes
        
    return SmartHomeDataset(data_dir, classes=classes)

In [11]:
datasets_info = ["HouseA", "CASAS", "DSADS", "PAMAP",]
def nothing(x):
    return x

for name in datasets_info:
    
    params = {
        "--results-dir": f"./Results/{name}/",
        "--data-dir": name.lower(),
        "--task-order": f"./Results/{name}/task_orders.txt",
    }

    p = [f"{k} {params[k]}" for k in params]
    p = " ".join(p)

    parser = arg_params.get_parser()
    args = parser.parse_args(p.split())

    d = select_dataset(args)
    
    d.set_target_tranform(nothing)
    print("Dataset name:", name)
    print("Overall", d.pddata.shape)
    SmartHomeDataset.detail(d.classes, [d], [d])
    
    print("\n")

Dataset name: HouseA
Overall (483, 15)
===== Smart Home dataset ======
  Activity Name  #Train  #Test  Total
0            A0      72     72    144
1            A1     165    165    330
2            A2      50     50    100
3            A3      78     78    156
4            A4      42     42     84
5            A5      76     76    152


Dataset name: CASAS
Overall (15655, 72)
===== Smart Home dataset ======
                  Activity Name  #Train  #Test  Total
0           R1_work_at_computer    4812   4812   9624
1           R2_work_at_computer    1480   1480   2960
2                      R1_sleep    2755   2755   5510
3                      R2_sleep    2716   2716   5432
4              R1_bed_to_toilet     163    163    326
5              R2_bed_to_toilet     107    107    214
6             R2_prepare_dinner    1184   1184   2368
7                   R2_watch_TV    1354   1354   2708
8              R2_prepare_lunch     563    563   1126
9  R1_work_at_dining_room_table     521    521   